In [74]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')
df = pd.read_csv("fin.csv", index_col=0)

In [75]:
df

,Software_Developer,Database_Administrator,Systems_Administrator,Project_manager,Web_Developer,Network_Administrator,Security_Analyst,Python_Developer,Java_Developer,Front_End_Developer,Ents,new
0,0,1,0,0,0,0,0,0,0,0,"[SQL, SSIS, OLTP, Backing, Generating, Log Shi...",sql ssis oltp backing generating log shipping ...
1,0,1,0,0,0,0,0,0,0,0,"[Microsoft, XP Microsoft, Assembly Language Mi...",microsoft xp microsoft assembly language micro...
2,0,1,0,0,0,0,0,0,0,0,"[Carrier Objective, Experienced Creating Users...",carrier objective experienced creating users r...
3,0,1,0,0,0,0,0,0,0,0,"[Amazon Redshift Administrator, yearsPostgresS...",amazon redshift administrator redshift adminis...
4,0,1,0,0,0,0,0,0,0,0,"[Scrum Master, Scrum Master, October April R...",scrum masterscrum master scrum master scrum ma...
...,...,...,...,...,...,...,...,...,...,...,...,...
29778,1,0,0,0,1,0,0,0,1,0,[ServiceNow DeveloperServiceNow DeveloperServi...,servicenow developerservicenow developerservic...
29779,1,0,0,0,1,0,0,0,0,0,"[DeveloperAndroid, DeveloperJoomla CMS, Samsun...",developerandroid developerjoomla cms samsung w...
29780,1,0,0,0,1,0,0,0,0,1,"[UI, WEB, DeveloperVisual, Web Applications De...",ui web developervisual web applications develo...
29781,1,0,0,0,1,0,0,0,0,0,"[September July , PHP]",php


In [76]:
classes = ['Software_Developer', 'Database_Administrator',
           'Systems_Administrator', 'Project_manager',
           'Web_Developer', 'Network_Administrator',
           'Security_Analyst', 'Python_Developer',
           'Java_Developer', 'Front_End_Developer']

In [77]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [78]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [79]:
tokenizer = Tokenizer(num_words=1000, lower = True)
tokenizer.fit_on_texts([str(i) for i in df['new'] ])
sequences = tokenizer.texts_to_sequences([str(i) for i in df['new'] ])
x = pad_sequences(sequences, maxlen=256)

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
x, df[df.columns[:10]], test_size=0.2, random_state=42, shuffle=True)

In [81]:
y_train.shape

(23826, 10)

In [86]:
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D

filter_length = 300

model = Sequential()
model.add(Embedding(5000, 100, input_length=256))
model.add(Dropout(0.25))
model.add(Conv1D(filter_length, 3, padding='same', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(500, activation = "sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(200, activation = "sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(100, activation = "sigmoid"))
model.add(Dropout(0.25))
model.add(Dense(10, activation = "sigmoid"))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

history = model.fit(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
745/745 [==============================] - 43s 54ms/step - loss: 0.3320 - auc_2: 0.8427
Epoch 2/30
745/745 [==============================] - 45s 61ms/step - loss: 0.2286 - auc_2: 0.9359
Epoch 3/30
745/745 [==============================] - 45s 60ms/step - loss: 0.1898 - auc_2: 0.9540
Epoch 4/30
745/745 [==============================] - 45s 60ms/step - loss: 0.1714 - auc_2: 0.9607
Epoch 5/30
745/745 [==============================] - 45s 61ms/step - loss: 0.1603 - auc_2: 0.9652
Epoch 6/30
745/745 [==============================] - 45s 61ms/step - loss: 0.1515 - auc_2: 0.9684
Epoch 7/30
745/745 [==============================] - 45s 61ms/step - loss: 0.1449 - auc_2: 0.9711
Epoch 8/30
745/745 [==============================] - 45s 61ms/step - loss: 0.1378 - auc_2: 0.9734
Epoch 9/30
745/745 [==============================] - 46s 62ms/step - loss: 0.1314 - auc_2: 0.9757
Epoch 10/30
745/745 [==============================] - 47s 63ms/step - loss: 0.1267 - auc_2: 0.9773
Epoch 11/

In [87]:
model.evaluate(X_test, y_test)

187/187 [==============================] - 3s 15ms/step - loss: 0.1952 - auc_2: 0.9609


[0.19515077769756317, 0.9609494209289551]

In [88]:
model.save("conv.h")

INFO:tensorflow:Assets written to: conv.h\assets
